# Analysis of Covariance

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Single-factor-Analysis" data-toc-modified-id="Single-factor-Analysis-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Single-factor Analysis</a></span><ul class="toc-item"><li><span><a href="#ANOVA-model" data-toc-modified-id="ANOVA-model-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>ANOVA model</a></span></li><li><span><a href="#ANCOVA-model" data-toc-modified-id="ANCOVA-model-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>ANCOVA model</a></span></li><li><span><a href="#Partial-F-test-for-treatment-effects" data-toc-modified-id="Partial-F-test-for-treatment-effects-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Partial F-test for treatment effects</a></span></li><li><span><a href="#Covariance-of-estimators" data-toc-modified-id="Covariance-of-estimators-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Covariance of estimators</a></span></li></ul></li><li><span><a href="#Two-factor--Analysis" data-toc-modified-id="Two-factor--Analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Two-factor  Analysis</a></span><ul class="toc-item"><li><span><a href="#Model-fitting" data-toc-modified-id="Model-fitting-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Model fitting</a></span></li><li><span><a href="#Covariance-matrix-of-estimators" data-toc-modified-id="Covariance-matrix-of-estimators-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Covariance matrix of estimators</a></span></li></ul></li></ul></div>

In [ ]:
print('\nEnabling interactive shell outputs ...')
print('   Use command pass; to disable cell text outputs')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings('ignore') 
warnings.simplefilter(action="ignore",category=UserWarning)
warnings.simplefilter(action="ignore",category=FutureWarning)

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm



%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

## Single-factor Analysis

In [ ]:
cracker_data = pd.read_excel('data/lect06-ancova.xlsx', sheet_name='Cracker')
cracker_data

In [ ]:
def codingfunc(x):
    if x == 1:
        return (1,0)
    elif x == 2:
        return (0,1)
    else:
        return (-1,-1)

cracker_data[['I1', 'I2']] = pd.DataFrame(cracker_data['Promotion'].apply(codingfunc).tolist(), 
                                           index=cracker_data.index)
cracker_data.PreviousSold -= cracker_data.PreviousSold.mean()

In [ ]:
cracker_data

### ANOVA model

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm


formula = 'PromotionSold ~ C(Promotion)'


cracker_lm = ols(formula, data=cracker_data).fit()
aov_table = anova_lm(cracker_lm) 
aov_table

### ANCOVA model 

In [ ]:
from statsmodels.formula.api import ols

formula = 'PromotionSold ~ I1 + I2 + PreviousSold'
cracker_lm = ols(formula, data=cracker_data).fit()
cracker_lm.summary2()

In [ ]:
print('MSE (ANCOVA): {:.2f}'.format(cracker_lm.mse_resid))

In [ ]:
def check_linreg_residuals(model):

    kws = dict(color='blue', marker='o', markersize=7, alpha=0.5)
    sns1_kws = dict(marker='o', s=70, alpha=0.5)
    sns2_kws = dict(marker='o', s=7, alpha=0.5)
    
    x = model.resid
    yhat = model.fittedvalues
    
    fig = plt.figure(figsize=(9, 7))

    ax1 = fig.add_subplot(221)
    ax2 = fig.add_subplot(222)
    ax3 = fig.add_subplot(223)
    ax4 = fig.add_subplot(224)

    ax1.scatter(yhat, x, **sns1_kws)
    ax1.set_title('Residuals vs. Fitted values')
    
    n=len(x)
    #sns.residplot(np.linspace(1,n,n), x, ax=ax2, scatter_kws=sns1_kws)
    #ax2.set_title('Sequence plot')
    ax2.scatter(yhat+x, yhat, **sns1_kws)
    ax2.set_title('Actual vs. Fitted values')
      
    # Box plot
    ax3.boxplot(x,showmeans=True)
    ax3.set_title('Boxplot')

    # qq plot
    sm.graphics.qqplot(x,line='q', ax=ax4, **kws)
    ax4.set_title('Normal Q-Q plot')
    plt.tight_layout()

In [ ]:
check_linreg_residuals(cracker_lm)

### Partial F-test for treatment effects

In [ ]:
cracker_reduced_lm = ols('PromotionSold ~ PreviousSold', data=cracker_data).fit()
cracker_reduced_lm.summary2()

In [ ]:
print('SSE_k (reduced model): {:.2f} '.format(cracker_reduced_lm.ssr))
print('SSE (full model): {:.2f} '.format(cracker_lm.ssr))
print('MSE (full model): {:.2f} '.format(cracker_lm.mse_resid))

In [ ]:
from scipy import stats
k=2
partial_F0 = ((cracker_reduced_lm.ssr-cracker_lm.ssr)/k)/cracker_lm.mse_resid
print('Test statistic: {:.2f}, P-value: {:.4f}'.format(partial_F0, 
                                                       stats.f.sf(partial_F0, k, cracker_lm.df_resid)))
print('Critical value at 0.05: {:.2f} '.format(stats.f.isf(0.05, k, cracker_lm.df_resid)))

### Covariance of estimators

In [ ]:
import statsmodels.api as sm
from numpy.linalg import inv

Xreg = sm.add_constant(cracker_data[['I1','I2','PreviousSold']]) 
Covmat = cracker_lm.mse_resid*inv(Xreg.T@Xreg)
np.set_printoptions(precision=4)
Covmat

## Two-factor  Analysis

In [ ]:
flower_data = pd.read_excel('data/lect06-ancova.xlsx', sheet_name='Flower')
flower_data

In [ ]:
def codingfunc(x):
    if x == 1:
        return 1
    else:
        return -1
      
flower_data['I1'] = flower_data['Variety'].apply(codingfunc)
flower_data['I2'] = flower_data['Moisture'].apply(codingfunc)
flower_data['I1I2'] = flower_data['I1']*flower_data['I2'] 
flower_data.Plot -= flower_data.Plot.mean()
flower_data

### Model fitting

In [ ]:
from statsmodels.formula.api import ols

formula = 'Sales ~ I1 + I2 + I1I2 + Plot'
flower_lm = ols(formula, data=flower_data).fit()
flower_lm.summary2()

In [ ]:
formula = 'Sales ~ I1 + I2 + Plot'
flower_lm = ols(formula, data=flower_data).fit()
flower_lm.summary2()

This example has one coefficient for each factor and the coded variables  
are uncorrelated. So the significance of a factor can be determined from 
that of the coefficient.


Significance of factor can be generally determined from partial f-tests.  
The reduced models are obtained by removing coefficients related to each  
factor at the time.

### Covariance matrix of estimators

In [ ]:
import statsmodels.api as sm
from numpy.linalg import inv

Xreg = sm.add_constant(flower_data[['I1', 'I2', 'I1I2', 'Plot']])
Covmat = flower_lm.mse_resid*inv(Xreg.T@Xreg)
np.set_printoptions(precision=4)
print(Covmat)